## 차량 데이터 전처리

- GWCT
    - GWCT_202009.csv
    - GWCT_202010.csv
    - GWCT_202011.csv
    - GWCT_202012.csv
    - GWCT_2021_first_half.csv
- KIT
    - KIT_202009.csv
    - KIT_202010.csv 
    - KIT_202011.csv
    - KIT_202012.csv
    - KIT_2021_first_half.csv

In [27]:
import pandas as pd
df = pd.read_csv('./data/terminal_operation/GWCT/GWCT_202009.csv', encoding="cp949")
print(df.shape)
df

(98681, 7)


,차량번호,진입게이트,진입일시,진출게이트,진출일시,작업시간,터미널
0,경남99바****,4번 GATE,2020-09-01 07:58,10번 GATE,2020-09-01 08:06,00:07:02,GWCT
1,경남99바****,2번 GATE,2020-09-01 07:59,8번 GATE,2020-09-01 08:24,00:24:19,GWCT
2,울산99바****,5번 GATE,2020-09-01 08:00,10번 GATE,2020-09-01 08:28,00:28:16,GWCT
3,광주99바****,2번 GATE,2020-09-01 08:00,8번 GATE,2020-09-01 08:44,00:44:36,GWCT
4,광주99사****,4번 GATE,2020-09-01 08:00,10번 GATE,2020-09-01 08:11,00:10:46,GWCT
...,...,...,...,...,...,...,...
98676,부산99사****,5번 GATE,2020-09-30 19:00,9번 GATE,2020-09-30 19:12,00:11:55,KIT
98677,전남80바****,4번 GATE,2020-09-30 19:02,9번 GATE,2020-09-30 19:13,00:11:03,KIT
98678,전남98바****,5번 GATE,2020-09-30 19:21,8번 GATE,2020-09-30 20:48,01:26:30,KIT
98679,전남98바****,4번 GATE,2020-09-30 19:55,9번 GATE,2020-09-30 20:55,00:59:57,KIT


In [30]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
display(df)

,차량번호,진입게이트,진입일시,진출게이트,진출일시,작업시간,터미널
0,경남99바****,4번 GATE,2020-09-01 07:58,10번 GATE,2020-09-01 08:06,00:07:02,GWCT
1,경남99바****,2번 GATE,2020-09-01 07:59,8번 GATE,2020-09-01 08:24,00:24:19,GWCT
2,울산99바****,5번 GATE,2020-09-01 08:00,10번 GATE,2020-09-01 08:28,00:28:16,GWCT
3,광주99바****,2번 GATE,2020-09-01 08:00,8번 GATE,2020-09-01 08:44,00:44:36,GWCT
4,광주99사****,4번 GATE,2020-09-01 08:00,10번 GATE,2020-09-01 08:11,00:10:46,GWCT
...,...,...,...,...,...,...,...
98676,부산99사****,5번 GATE,2020-09-30 19:00,9번 GATE,2020-09-30 19:12,00:11:55,KIT
98677,전남80바****,4번 GATE,2020-09-30 19:02,9번 GATE,2020-09-30 19:13,00:11:03,KIT
98678,전남98바****,5번 GATE,2020-09-30 19:21,8번 GATE,2020-09-30 20:48,01:26:30,KIT
98679,전남98바****,4번 GATE,2020-09-30 19:55,9번 GATE,2020-09-30 20:55,00:59:57,KIT


시간대별로 정리해야함. 

1. 진입 시간을 기준으로 시간대별로 그룹화 - 시간별로 차량 진입 횟수 집계

2. 진출 시간을 사용하여 항구에 머문 시간계산, 시간대별로 집계

3. 작업 시간계산 - 진출 시간 - 진입 시간 => 평균 작업 시간 계산

4. 시간대별로 집계 후 결측치 처리. 시간대별 평균 대체, 앞 뒤 시간대 데이터로 보간, 이상치 제거 혹은 수정

In [37]:

import pandas as pd
df = pd.read_csv('./data/terminal_operation/GWCT/GWCT_202009.csv', encoding="cp949")

# datetime 형식으로 변환
df['진입일시'] = pd.to_datetime(df['진입일시'])
df['진출일시'] = pd.to_datetime(df['진출일시'])


In [25]:
pd.set_option('display.max_rows', 500)

In [26]:
df

,차량번호,진입게이트,진입일시,진출게이트,진출일시,작업시간,터미널
0,경남99바****,4번 GATE,2020-09-01 07:58:00,10번 GATE,2020-09-01 08:06:00,00:07:02,GWCT
1,경남99바****,2번 GATE,2020-09-01 07:59:00,8번 GATE,2020-09-01 08:24:00,00:24:19,GWCT
2,울산99바****,5번 GATE,2020-09-01 08:00:00,10번 GATE,2020-09-01 08:28:00,00:28:16,GWCT
3,광주99바****,2번 GATE,2020-09-01 08:00:00,8번 GATE,2020-09-01 08:44:00,00:44:36,GWCT
4,광주99사****,4번 GATE,2020-09-01 08:00:00,10번 GATE,2020-09-01 08:11:00,00:10:46,GWCT
...,...,...,...,...,...,...,...
98676,부산99사****,5번 GATE,2020-09-30 19:00:00,9번 GATE,2020-09-30 19:12:00,00:11:55,KIT
98677,전남80바****,4번 GATE,2020-09-30 19:02:00,9번 GATE,2020-09-30 19:13:00,00:11:03,KIT
98678,전남98바****,5번 GATE,2020-09-30 19:21:00,8번 GATE,2020-09-30 20:48:00,01:26:30,KIT
98679,전남98바****,4번 GATE,2020-09-30 19:55:00,9번 GATE,2020-09-30 20:55:00,00:59:57,KIT


In [43]:
 # 작업시간을 timedelta 형식으로 변환

df['작업시간'] = pd.to_timedelta(df['작업시간'])

In [7]:
df

,차량번호,진입게이트,진입일시,진출게이트,진출일시,작업시간,터미널
0,경남99바****,4번 GATE,2020-09-01 07:58:00,10번 GATE,2020-09-01 08:06:00,0 days 00:07:02,GWCT
1,경남99바****,2번 GATE,2020-09-01 07:59:00,8번 GATE,2020-09-01 08:24:00,0 days 00:24:19,GWCT
2,울산99바****,5번 GATE,2020-09-01 08:00:00,10번 GATE,2020-09-01 08:28:00,0 days 00:28:16,GWCT
3,광주99바****,2번 GATE,2020-09-01 08:00:00,8번 GATE,2020-09-01 08:44:00,0 days 00:44:36,GWCT
4,광주99사****,4번 GATE,2020-09-01 08:00:00,10번 GATE,2020-09-01 08:11:00,0 days 00:10:46,GWCT
...,...,...,...,...,...,...,...
98676,부산99사****,5번 GATE,2020-09-30 19:00:00,9번 GATE,2020-09-30 19:12:00,0 days 00:11:55,KIT
98677,전남80바****,4번 GATE,2020-09-30 19:02:00,9번 GATE,2020-09-30 19:13:00,0 days 00:11:03,KIT
98678,전남98바****,5번 GATE,2020-09-30 19:21:00,8번 GATE,2020-09-30 20:48:00,0 days 01:26:30,KIT
98679,전남98바****,4번 GATE,2020-09-30 19:55:00,9번 GATE,2020-09-30 20:55:00,0 days 00:59:57,KIT


In [63]:
# 시간대별 차량 진입 횟수 계산

df['진입시간_시간대'] = df['진입일시'].dt.hour
hourly_entries = df.groupby('진입시간_시간대').size()
df

In [45]:
hourly_entries

진입시간_시간대
1        14
2        13
3         5
7       310
8      6396
9      8720
10     9856
11     8370
12      454
13    11252
14    11008
15    11311
16    10751
17     6804
18     1071
19     6401
20     3023
21     1843
22      895
23      184
dtype: int64

In [38]:
# 시간대별로 차량 쳬류 시간(진출일시 - 진입일시) 계산 그 다음 평균 계산

df['체류시간'] = (df['진출일시'] - df['진입일시']).dt.total_seconds()
df

,차량번호,진입게이트,진입일시,진출게이트,진출일시,작업시간,터미널,체류시간
0,경남99바****,4번 GATE,2020-09-01 07:58:00,10번 GATE,2020-09-01 08:06:00,00:07:02,GWCT,480.0
1,경남99바****,2번 GATE,2020-09-01 07:59:00,8번 GATE,2020-09-01 08:24:00,00:24:19,GWCT,1500.0
2,울산99바****,5번 GATE,2020-09-01 08:00:00,10번 GATE,2020-09-01 08:28:00,00:28:16,GWCT,1680.0
3,광주99바****,2번 GATE,2020-09-01 08:00:00,8번 GATE,2020-09-01 08:44:00,00:44:36,GWCT,2640.0
4,광주99사****,4번 GATE,2020-09-01 08:00:00,10번 GATE,2020-09-01 08:11:00,00:10:46,GWCT,660.0
...,...,...,...,...,...,...,...,...
98676,부산99사****,5번 GATE,2020-09-30 19:00:00,9번 GATE,2020-09-30 19:12:00,00:11:55,KIT,720.0
98677,전남80바****,4번 GATE,2020-09-30 19:02:00,9번 GATE,2020-09-30 19:13:00,00:11:03,KIT,660.0
98678,전남98바****,5번 GATE,2020-09-30 19:21:00,8번 GATE,2020-09-30 20:48:00,01:26:30,KIT,5220.0
98679,전남98바****,4번 GATE,2020-09-30 19:55:00,9번 GATE,2020-09-30 20:55:00,00:59:57,KIT,3600.0


In [46]:
hourly_average_stay = df.groupby('진입시간_시간대')['체류시간'].mean()
hourly_average_stay

진입시간_시간대
1     1748.571429
2      480.000000
3      432.000000
7     2660.129032
8     2121.838649
9     2169.805046
10    2454.125406
11    3880.172043
12    2078.590308
13    2262.392464
14    2392.881541
15    2663.472726
16    2789.729328
17    3192.292769
18    2610.308123
19    2780.096860
20    2341.786305
21    2006.467716
22    1749.653631
23    1179.782609
Name: 체류시간, dtype: float64

In [47]:
# 시간대별 평균 작업시간 구하기

hourly_average_work_time = df.groupby('진입시간_시간대')['작업시간'].mean().dt.total_seconds()
hourly_average_work_time

진입시간_시간대
1     1763.500000
2      483.461538
3      438.200000
7     2657.206452
8     2121.804253
9     2169.995528
10    2454.263596
11    3880.060215
12    2077.988987
13    2262.231426
14    2392.777525
15    2663.525683
16    2789.339224
17    3192.381540
18    2609.565826
19    2780.459147
20    2341.265961
21    2005.678242
22    1750.798883
23    1184.315217
Name: 작업시간, dtype: float64

In [48]:
print(hourly_entries.head())
print(hourly_average_stay.head())
print(hourly_average_work_time.head())

진입시간_시간대
1      14
2      13
3       5
7     310
8    6396
dtype: int64
진입시간_시간대
1    1748.571429
2     480.000000
3     432.000000
7    2660.129032
8    2121.838649
Name: 체류시간, dtype: float64
진입시간_시간대
1    1763.500000
2     483.461538
3     438.200000
7    2657.206452
8    2121.804253
Name: 작업시간, dtype: float64


In [4]:
import pandas as pd
from glob import glob

# 파일 경로 패턴을 사용하여 모든 CSV 파일의 목록을 생성합니다.
file_paths = glob('./data/terminal_operation/GWCT//*.csv')+ glob('./data/terminal_operation/KIT//*.csv')

In [2]:
for path in file_paths:
    print(path)

./data/terminal_operation/GWCT\GWCT_202009.csv
./data/terminal_operation/GWCT\GWCT_202010.csv
./data/terminal_operation/GWCT\GWCT_202011.csv
./data/terminal_operation/GWCT\GWCT_202012.csv
./data/terminal_operation/GWCT\GWCT_2021_first_half.csv
./data/terminal_operation/KIT\KIT_202009.csv
./data/terminal_operation/KIT\KIT_202010.csv
./data/terminal_operation/KIT\KIT_202011.csv
./data/terminal_operation/KIT\KIT_202012.csv
./data/terminal_operation/KIT\KIT_2021_first_half.csv


In [5]:
import pandas as pd
from glob import glob

# 파일 경로 확인
print("File paths:")
print(file_paths)

all_data = pd.DataFrame()
for file_path in file_paths:
    try:
        monthly_data = pd.read_csv(file_path, encoding='utf-8')  #
    except UnicodeDecodeError:
        monthly_data = pd.read_csv(file_path, encoding='cp949')  
        
    # 파일 내용 확인
    print(f"Data in {file_path}:")
    print(monthly_data.head())

    monthly_data['진입일시'] = pd.to_datetime(monthly_data['진입일시'])
    monthly_data['진출일시'] = pd.to_datetime(monthly_data['진출일시'])

    monthly_data['체류시간'] = (monthly_data['진출일시'] - monthly_data['진입일시'])

    # '작업시간' 열의 데이터 형식 확인
    # print(f"Data format in '작업시간' column in {file_path}:")
    # print(monthly_data['작업시간'].head())

    # 필요한 경우, 데이터 형식에 맞는 변환 로직 적용
    # 예: monthly_data['작업시간'] = monthly_data['작업시간'].apply(custom_format_function)

    # monthly_data['작업시간'] = pd.to_timedelta(monthly_data['작업시간'])
    # if monthly_data['작업시간'].dtype == 'object':
    #     monthly_data['작업시간'] = pd.to_timedelta(monthly_data['작업시간'])

    all_data = pd.concat([all_data, monthly_data], ignore_index=True)

File paths:
['./data/terminal_operation/GWCT\\GWCT_202009.csv', './data/terminal_operation/GWCT\\GWCT_202010.csv', './data/terminal_operation/GWCT\\GWCT_202011.csv', './data/terminal_operation/GWCT\\GWCT_202012.csv', './data/terminal_operation/GWCT\\GWCT_2021_first_half.csv', './data/terminal_operation/KIT\\KIT_202009.csv', './data/terminal_operation/KIT\\KIT_202010.csv', './data/terminal_operation/KIT\\KIT_202011.csv', './data/terminal_operation/KIT\\KIT_202012.csv', './data/terminal_operation/KIT\\KIT_2021_first_half.csv']
Data in ./data/terminal_operation/GWCT\GWCT_202009.csv:
        차량번호    진입게이트              진입일시     진출게이트              진출일시      작업시간  \
0  경남99바****  4번 GATE  2020-09-01 07:58  10번 GATE  2020-09-01 08:06  00:07:02   
1  경남99바****  2번 GATE  2020-09-01 07:59   8번 GATE  2020-09-01 08:24  00:24:19   
2  울산99바****  5번 GATE  2020-09-01 08:00  10번 GATE  2020-09-01 08:28  00:28:16   
3  광주99바****  2번 GATE  2020-09-01 08:00   8번 GATE  2020-09-01 08:44  00:44:36   
4  광주99사

In [6]:
# 병합된 데이터 프레임 확인
print("Combined Data Frame:")
print(all_data.head())

# datetime 변환 확인
print("Data types of '진입일시' and '진출일시' in the combined data frame:")
print(all_data['진입일시'].dtype)
print(all_data['진출일시'].dtype)

Combined Data Frame:
        차량번호    진입게이트                진입일시     진출게이트                진출일시  \
0  경남99바****  4번 GATE 2020-09-01 07:58:00  10번 GATE 2020-09-01 08:06:00   
1  경남99바****  2번 GATE 2020-09-01 07:59:00   8번 GATE 2020-09-01 08:24:00   
2  울산99바****  5번 GATE 2020-09-01 08:00:00  10번 GATE 2020-09-01 08:28:00   
3  광주99바****  2번 GATE 2020-09-01 08:00:00   8번 GATE 2020-09-01 08:44:00   
4  광주99사****  4번 GATE 2020-09-01 08:00:00  10번 GATE 2020-09-01 08:11:00   

       작업시간   터미널            체류시간  
0  00:07:02  GWCT 0 days 00:08:00  
1  00:24:19  GWCT 0 days 00:25:00  
2  00:28:16  GWCT 0 days 00:28:00  
3  00:44:36  GWCT 0 days 00:44:00  
4  00:10:46  GWCT 0 days 00:11:00  
Data types of '진입일시' and '진출일시' in the combined data frame:
datetime64[ns]
datetime64[ns]


In [9]:
# 진입일시의 날짜와 시간 추출
all_data['날짜'] = all_data['진입일시'].dt.date
all_data['시간대'] = all_data['진입일시'].dt.hour

# 날짜별, 시간대별 차량 진입 횟수
daily_hourly_truck_count = all_data.groupby(['날짜','시간대']).size()

# 날짜별, 시간대별 체류시간 평균계산
daily_hourly_stay = all_data.groupby(['날짜','시간대'])['체류시간'].mean()

print("[[시간대별 진입 횟수]]")
print(daily_hourly_truck_count)
print("[[평균 체류 시간]]")
print(daily_hourly_stay)

[[시간대별 진입 횟수]]
날짜          시간대
2020-09-01  7        8
            8      520
            9      754
            10     900
            11     898
                  ... 
2021-06-30  19     132
            20      91
            21      49
            22      16
            23       4
Length: 5339, dtype: int64
[[평균 체류 시간]]
날짜          시간대
2020-09-01  7               0 days 00:21:00
            8     0 days 00:30:33.230769230
            9     0 days 00:29:29.761273209
            10    0 days 00:29:26.533333333
            11    0 days 00:58:36.881959910
                             ...           
2021-06-30  19    0 days 00:36:09.545454545
            20    0 days 00:25:24.318681318
            21    0 days 00:35:33.816326530
            22       0 days 00:16:34.875000
            23       0 days 00:20:26.500000
Name: 체류시간, Length: 5339, dtype: timedelta64[ns]


In [10]:
daily_hourly_truck_count = daily_hourly_truck_count.reset_index()
daily_hourly_truck_count.columns = ['날짜', '시간대', '차량진입횟수']
daily_hourly_truck_count

,날짜,시간대,차량진입횟수
0,2020-09-01,7,8
1,2020-09-01,8,520
2,2020-09-01,9,754
3,2020-09-01,10,900
4,2020-09-01,11,898
...,...,...,...
5334,2021-06-30,19,132
5335,2021-06-30,20,91
5336,2021-06-30,21,49
5337,2021-06-30,22,16


In [11]:
daily_hourly_stay = daily_hourly_stay.reset_index()
daily_hourly_stay

,날짜,시간대,체류시간
0,2020-09-01,7,0 days 00:21:00
1,2020-09-01,8,0 days 00:30:33.230769230
2,2020-09-01,9,0 days 00:29:29.761273209
3,2020-09-01,10,0 days 00:29:26.533333333
4,2020-09-01,11,0 days 00:58:36.881959910
...,...,...,...
5334,2021-06-30,19,0 days 00:36:09.545454545
5335,2021-06-30,20,0 days 00:25:24.318681318
5336,2021-06-30,21,0 days 00:35:33.816326530
5337,2021-06-30,22,0 days 00:16:34.875000


In [12]:
# dasily_hourly_truck_count와 daily_hourly_stay 합쳐서 파일로 저장
combined_data = pd.merge(daily_hourly_truck_count, 
                         daily_hourly_stay,
                         on=['날짜','시간대'])
combined_data.head()

,날짜,시간대,차량진입횟수,체류시간
0,2020-09-01,7,8,0 days 00:21:00
1,2020-09-01,8,520,0 days 00:30:33.230769230
2,2020-09-01,9,754,0 days 00:29:29.761273209
3,2020-09-01,10,900,0 days 00:29:26.533333333
4,2020-09-01,11,898,0 days 00:58:36.881959910


In [13]:
combined_data.tail(20)

,날짜,시간대,차량진입횟수,체류시간
5319,2021-06-30,0,2,0 days 00:16:37
5320,2021-06-30,1,1,0 days 00:16:16
5321,2021-06-30,6,2,0 days 00:26:34
5322,2021-06-30,7,29,0 days 00:58:16.724137931
5323,2021-06-30,8,242,0 days 00:50:51.714876033
5324,2021-06-30,9,273,0 days 00:40:25.454212454
5325,2021-06-30,10,344,0 days 00:50:33.302325581
5326,2021-06-30,11,329,0 days 00:59:55.711246200
5327,2021-06-30,12,83,0 days 00:59:19.204819277
5328,2021-06-30,13,393,0 days 00:47:22.483460559


In [18]:
# 병합된 데이터를 CSV 파일로 저장
combined_data.to_csv('./filtered_data/truck.csv', index=False)

print('saved truck.csv')

saved truck.csv


In [19]:
df = pd.read_csv('./filtered_data/truck.csv')
df

,날짜,시간대,차량진입횟수,체류시간
0,2020-09-01,7,8,0 days 00:21:00
1,2020-09-01,8,520,0 days 00:30:33.230769230
2,2020-09-01,9,754,0 days 00:29:29.761273209
3,2020-09-01,10,900,0 days 00:29:26.533333333
4,2020-09-01,11,898,0 days 00:58:36.881959910
...,...,...,...,...
5334,2021-06-30,19,132,0 days 00:36:09.545454545
5335,2021-06-30,20,91,0 days 00:25:24.318681318
5336,2021-06-30,21,49,0 days 00:35:33.816326530
5337,2021-06-30,22,16,0 days 00:16:34.875000


In [ ]:
#  20시 40분 차량 데이터 필터링 완료